In [ ]:
import os
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy.interpolate import interp2d
from scipy.signal import find_peaks
from scipy.stats import zscore
import pickle
import os
from scipy.interpolate import griddata
import logging
import sys 
import shutil
from bisect import bisect_left
from ast import literal_eval
import seaborn as sns
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer
from IPython.display import display
from ipyfilechooser import FileChooser
from datetime import datetime
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [ ]:
path=f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_CGP_analysis/Osc_2025-05-28_14_08_56_fineDetectionGLM_pynapple/"

Mouse= 'ThreeColDots'
Data='Ca'
Osc='Spdl'
Ctx='S1'
Drug='baseline'
Coupling=''

In [ ]:
filename=f"{path}/{Osc}_{Data}PSTH_{Coupling}PFC{Drug}_{Mouse}.pkl"
with open(filename, 'rb') as pickle_file:
    dataPFC = pickle.load(pickle_file)
filename=f"{path}/{Osc}_{Data}PSTH_{Coupling}S1{Drug}_{Mouse}.pkl"
with open(filename, 'rb') as pickle_file:
    dataS1 = pickle.load(pickle_file)
filename=f"{path}/{Osc}_{Data}PSTH_{Coupling}S1PFC{Drug}_{Mouse}.pkl"
with open(filename, 'rb') as pickle_file:
    dataS1PFC = pickle.load(pickle_file)

In [ ]:
nr='ThreeColDots395'

In [ ]:
def process_and_plot(data, ax, vmin=0, vmax=1):
    #data1=data.iloc[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    data1=data[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    n_cols = np.shape(data1)[1] // 4  # Floor division to get the number of columns in 1/4th
    baseline_columns = data1[:, :n_cols]  # Select the first 'n_cols' columns
    mean_baseline = np.nanmean(baseline_columns,axis=1)
    data2 = data1-mean_baseline[:, np.newaxis]
    sns.heatmap(data2, ax=ax, cmap='viridis', xticklabels=False, yticklabels=False, vmin=vmin, vmax=vmax)
    #sns.heatmap(data2, ax=ax, cmap='viridis', xticklabels=False, yticklabels=False, vmin=vmin, vmax=vmax)
plt.close()

fig, axes = plt.subplots(1, 3, figsize=(6, 2))

# Check if subplots are correctly created
process_and_plot(dataPFC[nr], axes[0])
process_and_plot(dataS1[nr], axes[1])
process_and_plot(dataS1PFC[nr], axes[2])

#plt.savefig(f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/Extract_{nr}_heatmap_Spdl_Catraces.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()


In [ ]:
plt.close()
fig, axes = plt.subplots(1, 3, figsize=(4, 2))  # Increased figure size for better readability

cmap = sns.light_palette("#008B8B", as_cmap=True)
#cmap = sns.light_palette("black", as_cmap=True)

def plot_lines(data, ax, title, xlabel=None, title_color='black', cmap=cmap):
    #data1=data.iloc[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    data1=data[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    n_cols = np.shape(data1)[1]// 4  # Floor division to get the number of columns in 1/4th
    baseline_columns = data1[:, :n_cols]  # Select the first 'n_cols' columns
    mean_baseline = np.nanmean(baseline_columns,axis=1)
    data2 = data1-mean_baseline[:, np.newaxis]
    data2=np.array(data2)
    num_lines = data2.shape[0]
    #data=zscore(data, axis=0)  # Z-score normalization across rows
    colors = cmap(np.linspace(0., .8, num_lines))
    ax.text(-1,0, f'#1', color='black', ha='right', va='center', fontsize=8) 
    # Plot each line with a label
    for i in range(num_lines):
        #line_data = data2[i] - i * 1.5
        line_data = data[i] - i * 1.5
        ax.plot(0, min(line_data), line_data, color=colors[i], linewidth=1)
    ax.text(-1, min(line_data), f'#{i+1}', color='black', ha='right', va='center', fontsize=8) 
    ax.set_title(title, fontsize=12, color=title_color, pad=10)  # Added padding for the title
    if xlabel:
        ax.set_xlabel(xlabel, labelpad=2, fontsize=10)    
    ax.axis('off')  # Hide axis lines and labels

data = dataPFC[nr][::]*1   #[::2] Taking every other row
plot_lines(data, axes[0], 'PFC only', title_color='mediumseagreen')
data = dataS1[nr][::]*1
plot_lines(data, axes[1], 'S1 only', title_color='rebeccapurple')
data = dataS1PFC[nr][::]*1
plot_lines(data, axes[2], 'S1&PFC', title_color='orange')


output_path = f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure3_revised/Extract_line_{nr}_Spdl_Catraces.svg'
plt.savefig(output_path, format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()


In [ ]:
plt.close()
fig, axes = plt.subplots(1, 3, figsize=(4, 1))  # Increased figure size for better readability

#cmap = sns.light_palette("#008B8B", as_cmap=True)
cmap = sns.light_palette("black", as_cmap=True)

def plot_lines(data, ax,title_color='black', cmap=cmap):
    data1=data[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    n_cols = np.shape(data1)[1]// 4  # Floor division to get the number of columns in 1/4th
    baseline_columns = data1[:, :n_cols]   # Select the first 'n_cols' columns
    mean_baseline = np.nanmean(baseline_columns,axis=1)
    data2 = data1-mean_baseline[:, np.newaxis]
    ax.plot(np.nanmean(data2, axis=0), color=title_color, linewidth=2)
    ax.set_ylim(-.6, 1)    
    ax.axis('off')  # Hide axis lines and labels


data = dataPFC[nr]*10
plot_lines(data, axes[0], title_color='mediumseagreen')
data = dataS1[nr]*10 # [::2] Taking every other row
plot_lines(data, axes[1],  title_color='rebeccapurple')
data = dataS1PFC[nr]*10
plot_lines(data, axes[2],  title_color='orange')

output_path = f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure3_revised/Extract_AVG_{nr}_Spdl_Catraces.svg'
plt.savefig(output_path, format='svg', bbox_inches='tight', pad_inches=0, transparent=True)

# Show the plot
plt.show()
